## Import

In [3]:
import os
import datetime as dt
import pandas as pd
import numpy as np

In [2]:
# Import custom modules - NEED WINDOWS OS
#from CoEQWAL.imports import AuxFunctions as af, cs3, csPlots, cs_util as util, dss3_functions_reference as dss
import AuxFunctions as af, cs3, csPlots, cs_util as util, dss3_functions_reference as dss

## Define file names and ranges for DSS and vars

In [6]:
# list of study names file, tab, range, and extension
DssList = r'../dss_may_2024/Exploratory_Results_v03/callite_expl03_run_listing.xlsx'
DssTab = 'callite_expl03_run_listing'
DssMin = 'A1'
DssMax = 'A601'
DssExt = '_DV.dss'
DssOut = 'EDA_07_01_24_dss_names.csv'

# list of relevant variables file, tab, and range (B & C parts)
VarList = r'../Selected Studies and Variables/May27_variables.xlsx'
VarTab = 'May EDA'
VarMin = 'B1'
VarMax = 'C51'
VarOut = 'EDA_vars_07_01_24.csv'

# launch file path to (or CalSim) studies and output csv name
StudyDir = r'../dss_may_2024/Exploratory_Results_v03'
DataDir = r'../data/extract/'
DataOut = 'EDA_data_07_01_24.csv'

# WY csv
WYcsv = 'wytypestable.csv'

# time intervals
begYear = 1921
begMonth = 10
begDay = 31
endYear = 2003
endMonth = 9
endDay = 30

## Read names and construct study name

In [4]:
dsshdr, dssname = af.read_from_excel(DssList, DssTab, DssMin, DssMax, hdr=True)
dss_names = []
for i in range(len(dssname)):
    dss_names.append(dssname[i][0] + DssExt)
dss_names

['expl0000_DV.dss',
 'expl0001_DV.dss',
 'expl0002_DV.dss',
 'expl0003_DV.dss',
 'expl0004_DV.dss',
 'expl0005_DV.dss',
 'expl0006_DV.dss',
 'expl0007_DV.dss',
 'expl0008_DV.dss',
 'expl0009_DV.dss',
 'expl0010_DV.dss',
 'expl0011_DV.dss',
 'expl0012_DV.dss',
 'expl0013_DV.dss',
 'expl0014_DV.dss',
 'expl0015_DV.dss',
 'expl0016_DV.dss',
 'expl0017_DV.dss',
 'expl0018_DV.dss',
 'expl0019_DV.dss',
 'expl0020_DV.dss',
 'expl0021_DV.dss',
 'expl0022_DV.dss',
 'expl0023_DV.dss',
 'expl0024_DV.dss',
 'expl0025_DV.dss',
 'expl0026_DV.dss',
 'expl0027_DV.dss',
 'expl0028_DV.dss',
 'expl0029_DV.dss',
 'expl0030_DV.dss',
 'expl0031_DV.dss',
 'expl0032_DV.dss',
 'expl0033_DV.dss',
 'expl0034_DV.dss',
 'expl0035_DV.dss',
 'expl0036_DV.dss',
 'expl0037_DV.dss',
 'expl0038_DV.dss',
 'expl0039_DV.dss',
 'expl0040_DV.dss',
 'expl0041_DV.dss',
 'expl0042_DV.dss',
 'expl0043_DV.dss',
 'expl0044_DV.dss',
 'expl0045_DV.dss',
 'expl0046_DV.dss',
 'expl0047_DV.dss',
 'expl0048_DV.dss',
 'expl0049_DV.dss',


## Write dss names to file

In [5]:
dss_df = (pd.DataFrame(dss_names))
dss_df.to_csv(os.path.join(DataDir, DssOut))

## Read variables

In [6]:
hdr, vars = af.read_from_excel(VarList, VarTab,VarMin,VarMax,hdr=True)
var_df = pd.DataFrame(data=vars, columns=hdr)
var_df

,DSS Part B,DSS Part C
0,S_SHSTA,STORAGE
1,S_OROVL,STORAGE
2,S_TRNTY,STORAGE
3,S_MELON,STORAGE
4,S_MLRTN,STORAGE
5,S_FOLSM,STORAGE
6,S_SHSTALEVEL1DV,STORAGE-LEVEL
7,S_SHSTALEVEL5DV,STORAGE-LEVEL
8,S_OROVLLEVEL1DV,STORAGE-LEVEL
9,S_TRNTYLEVEL1DV,STORAGE-LEVEL


In [7]:
var_df.to_csv(os.path.join(DataDir,VarOut))

## Create datasets across studies


In [8]:
def preprocess_data_dss(df, begYear, begMonth, begDay, endYear, endMonth, endDay, addsl=False, addres = True, addpump = True, adddelcvp = True, adddelcvpag = True, addcvpscex = True, addcvpprf = True, adddelcvpswp = True):
    dvar_list = []
    combined_df = pd.DataFrame()
    
    for i, r in df.iterrows():
        dvar_list.append(f'/{r["DSS Part B"]}/{r["DSS Part C"]}/')

    for dss_name in dss_names:

        print(dss_name)

        # Create a blank python "calsim" object
        thiscs3 = cs3.calsim()

        # add start and end dates
        thiscs3.StartDate = dt.datetime(begYear,begMonth,begDay)
        thiscs3.EndDate = dt.datetime(endYear,endMonth,endDay)

        # add path to DSS (CURRENTLY EXPECTING DSS IN THE BASE DIRECTORY, BUT WE SHOULD MAKE IT A PARAMETER)
        DSS_FP = os.path.join(StudyDir, dss_name)
        thiscs3.DV_FP = DSS_FP

        # Retrieve the DSS data variables from the DSS file
        thiscs3.DVdata = cs3.csDVdata(thiscs3)
        thiscs3.DVdata.getDVts(filter=dvar_list)

        df = thiscs3.DVdata.DVtsDF.copy(deep=True)

        # if storage add the 2 variables to create a new one
        # Add S_SLSCVP and S_SLSWP into S_SLTOT

        if addsl:
            df[('CALLITE', 'S_SLTOT', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_SLCVP', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_SLSWP', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)

        if addpump:
            #df[('CALLITE', 'D_JONES_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'D_JONES', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'D_BANKS_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'D_BANKS', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'D_TOTAL_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'D_JONES_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'D_BANKS_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'D_TOTAL', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'D_JONES', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'D_BANKS', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addres:
            df[('CALLITE', 'S_RESTOT', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_OROVL', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_MELON', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_SHSTA', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_MLRTN', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_FOLSM', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_TRNTY', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'S_RESTOT_NOD', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_OROVL', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_SHSTA', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_TRNTY', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_FOLSM', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF') ]].sum(axis=1)

        if adddelcvp:
            #df[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_TOTAL_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if adddelcvpswp:
            #df[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVPSWP_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_SWP_TOTAL', 'DELIVERY-SWP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if adddelcvpag:
            #df[('CALLITE', 'DEL_CVP_PAG_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PAG_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PAG_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PAG_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PAG_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PAG_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PAG_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PAG_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PAG_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PAG_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addcvpscex:
            #df[('CALLITE', 'DEL_CVP_PSC_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PSC_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PEX_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PEX_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PSCEX_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PSC_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PEX_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PSCEX_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PSC_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PEX_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addcvpprf:
            #df[('CALLITE', 'DEL_CVP_PRF_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PRF_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PRF_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PRF_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PRF_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PRF_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PRF_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PRF_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PRF_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PRF_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        new_columns = [(col[0], f'{col[1]}_{dss_name[:-7]}', *col[2:]) if len(col) > 1 else (col[0], '') for col in df.columns]
        df.columns = pd.MultiIndex.from_tuples(new_columns)
        df.columns.names = ['A', 'B', 'C', 'D', 'E', 'F', 'Units']
        combined_df = pd.concat([combined_df, df], axis=1)

    return combined_df

In [9]:
df = preprocess_data_dss(var_df, begYear, begMonth, begDay, endYear, endMonth, endDay)
df.head(5)

expl0000_DV.dss
Opened file: ../dss_may_2024/Exploratory_Results_v03\expl0000_DV.dss
Valid catalog found: True 
Number of records: 39186
Took 0.05485081672668457 sec to convert paths to dataframe
Took 0.00997304916381836 sec to do group-by operation
Took 0.014961481094360352 sec to add column


C:\Dino\COEQWAL\GitMay2024\CoEQWAL\notebooks\AuxFunctions.py:456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['search_col'] = ['/'.join([str(c1) for c1 in c]) if type(c)==tuple else c for c in condcat.index]


Took 0.28125691413879395 sec to rebuild modified path
Filtering path list


C:\Dino\COEQWAL\GitMay2024\CoEQWAL\notebooks\AuxFunctions.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['full_path'] = ['/'+'/'.join([str(c[1][k]) if k !='' else '' for k in keep_parts])+'/' for c in condcat.iterrows()]


expl0001_DV.dss=============================================-] 97.9% ...Retrieving 47 DV Time Series
Opened file: ../dss_may_2024/Exploratory_Results_v03\expl0001_DV.dss
Valid catalog found: True 
Number of records: 39204
Took 0.05388045310974121 sec to convert paths to dataframe
Took 0.006015300750732422 sec to do group-by operation
Took 0.001995086669921875 sec to add column


...

expl0599_DV.dss=============================================-] 97.9% ...Retrieving 47 DV Time Series
Opened file: ../dss_may_2024/Exploratory_Results_v03\expl0599_DV.dss
Valid catalog found: True 
Number of records: 39159
Took 0.061809539794921875 sec to convert paths to dataframe
Took 0.007979154586791992 sec to do group-by operation
Took 0.0029921531677246094 sec to add column


C:\Dino\COEQWAL\GitMay2024\CoEQWAL\notebooks\AuxFunctions.py:456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['search_col'] = ['/'.join([str(c1) for c1 in c]) if type(c)==tuple else c for c in condcat.index]


Took 0.28826212882995605 sec to rebuild modified path
Filtering path list


C:\Dino\COEQWAL\GitMay2024\CoEQWAL\notebooks\AuxFunctions.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condcat['full_path'] = ['/'+'/'.join([str(c[1][k]) if k !='' else '' for k in keep_parts])+'/' for c in condcat.iterrows()]


A                    CALLITE                                           \
B          C_SJRVER_expl0000 C_WILKNS_expl0000 DEL_CVP_PAG_N_expl0000   
C               FLOW-CHANNEL      FLOW-CHANNEL           DELIVERY-CVP   
D                       1MON              1MON                   1MON   
E                   2020D09E          2020D09E               2020D09E   
F                   PER-AVER          PER-AVER               PER-AVER   
Units                    CFS               CFS                    CFS   
1921-10-31       2505.279053       5427.564453              79.622688   
1921-11-30       1977.846924       4904.451660               0.000000   
1921-12-31       2740.872803       9056.176758               0.000000   
1922-01-31       2577.651367       6841.230469               0.000000   
1922-02-28       5592.397949      16573.013672               0.000000   

A                                                         \
B          DEL_CVP_PAG_S_expl0000 DEL_CVP_PEX_S_expl0000   
C                    DELIVERY-CVP           DELIVERY-CVP   
D                            1MON                   1MON   
E                        2020D09E               2020D09E   
F                        PER-AVER               PER-AVER   
Units                         CFS                    CFS   
1921-10-31             922.124268             978.930847   
1921-11-30             703.339600             336.382050   
1921-12-31             973.176086             147.935471   
1922-01-31            1708.295288             153.597214   
1922-02-28            2154.230957             442.686554   

A                                                         \
B          DEL_CVP_PMI_N_expl0000 DEL_CVP_PMI_S_expl0000   
C                    DELIVERY-CVP           DELIVERY-CVP   
D                            1MON                   1MON   
E                        2020D09E               2020D09E   
F                        PER-AVER               PER-AVER   
Units                         CFS                    CFS   
1921-10-31             175.152054             177.974594   
1921-11-30             137.904861             239.364700   
1921-12-31             122.706566             237.118301   
1922-01-31             119.161469             156.204773   
1922-02-28             131.604660              80.271103   

A                                                         \
B          DEL_CVP_PRF_N_expl0000 DEL_CVP_PRF_S_expl0000   
C                    DELIVERY-CVP           DELIVERY-CVP   
D                            1MON                   1MON   
E                        2020D09E               2020D09E   
F                        PER-AVER               PER-AVER   
Units                         CFS                    CFS   
1921-10-31             330.147858            1070.540649   
1921-11-30             268.888885             689.739258   
1921-12-31             157.755371             314.705475   
1922-01-31              74.811829             141.364426   
1922-02-28              68.422615             106.018906   

A                                  ...                                      \
B          DEL_CVP_PSC_N_expl0000  ... WYT_TRIN__expl0599  X2_PRV_expl0599   
C                    DELIVERY-CVP  ...    WATER-YEAR-TYPE X2-POSITION-PREV   
D                            1MON  ...               1MON             1MON   
E                        2020D09E  ...           2020D09E         2020D09E   
F                        PER-AVER  ...           PER-AVER         PER-AVER   
Units                         CFS  ...               NONE               KM   
1921-10-31             776.821716  ...                3.0        68.202873   
1921-11-30             990.790405  ...                3.0        67.414101   
1921-12-31               0.000000  ...                3.0        73.777863   
1922-01-31              24.395163  ...                3.0        73.254784   
1922-02-28               0.000000  ...                3.0        70.161186   

A                      

## Write dataframes to file

In [10]:
df.to_csv(os.path.join(DataDir, DataOut), header=True, index=True)

### Separate Into Dry & Wet Years

In [11]:
def add_water_year_column(df):
    df['Date'] = pd.to_datetime(df.index)
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['WaterYear'] = np.where(df['Month'] >= 10, df['Year'] + 1, df['Year'])
    return df.drop(["Date", "Year", "Month"], axis = 1)

In [ ]:
df = add_water_year_column(df)

In [4]:
wy_value_map = {
    1: "Wet",
    2: "Wet",
    3: "Wet",
    4: "Dry",
    5: "Dry",
}

In [7]:
df_wy = pd.read_csv(DataDir + WYcsv)
df_wy = df_wy[["SACindex_0", "WATERYEAR"]]
df_wy

,SACindex_0,WATERYEAR
0,2,1920
1,2,1921
2,2,1922
3,3,1923
4,5,1924
...,...,...
79,1,1999
80,2,2000
81,4,2001
82,2,2003


In [ ]:
year_to_category = df_wy.set_index('WATERYEAR')['SACindex_0'].map(wy_value_map).to_dict()
df['Category'] = df['WaterYear'].map(year_to_category)

wet_df = df[df['Category'] == "Wet"]
dry_df = df[df['Category'] == "Dry"]

df.drop(columns=['Category'], inplace=True)
wet_df.drop(columns=['Category'], inplace=True)
dry_df.drop(columns=['Category'], inplace=True)

In [17]:
wet_df.to_csv(os.path.join(DataDir, "wet_" + DataOut), header=True, index=True)

In [18]:
dry_df.to_csv(os.path.join(DataDir, "dry_" + DataOut), header=True, index=True)

## End of data extaction